In [46]:
import os
current_directory = os.getcwd()
print(current_directory)

/Users/macbookpro/hdd/MSc/Dissertation/multilabeltextclassification


In [47]:
import numpy
numpy.version.version

'1.22.0'

In [48]:
# importing libraries

from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
#from keras.preprocessing.sequence import pad_sequences
#from keras_preprocessing.sequence import pad_sequences
#from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils import pad_sequences
from tensorflow.keras.preprocessing import text, sequence
from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers
from tensorflow.python.keras.models import Model, Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Input, LSTM, Embedding, Dropout, SpatialDropout1D, Activation, SimpleRNN
from tensorflow.keras.layers import Conv1D, Bidirectional, GlobalMaxPool1D, MaxPooling1D, BatchNormalization
from tensorflow.keras.optimizers import Adam
#from tensorflow.keras.optimizers import SGD

# For custom metrics
import keras.backend as K
from keras.utils.vis_utils import plot_model
from keras.callbacks import EarlyStopping 

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
plt.style.use('seaborn')

import seaborn as sns
from IPython.display import Image

from tqdm import tqdm
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer 
import os, re, csv, math, codecs
from nltk.tokenize import word_tokenize
import string
import gensim

sns.set_style("whitegrid")
np.random.seed(0)

In [49]:
df = pd.read_csv('/Users/macbookpro/hdd/MSc/Dissertation/multilabeltextclassification/githubissuedata.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226163 entries, 0 to 226162
Data columns (total 17 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   title             225152 non-null  object
 1   body              225866 non-null  object
 2   task              226163 non-null  int64 
 3   bug               226163 non-null  int64 
 4   documentation     226163 non-null  int64 
 5   duplicate         226163 non-null  int64 
 6   enhancement       226163 non-null  int64 
 7   good_first_issue  226163 non-null  int64 
 8   help_wanted       226163 non-null  int64 
 9   invalid           226163 non-null  int64 
 10  question          226163 non-null  int64 
 11  wontfix           226163 non-null  int64 
 12  gitalk            226163 non-null  int64 
 13  priority_medium   226163 non-null  int64 
 14  priority_high     226163 non-null  int64 
 15  feature_request   226163 non-null  int64 
 16  feature           226163 non-null  int

In [50]:
df.head(10)

,title,body,task,bug,documentation,duplicate,enhancement,good_first_issue,help_wanted,invalid,question,wontfix,gitalk,priority_medium,priority_high,feature_request,feature
0,My Account Paid laptop 1440 resolution Updat...,Case:Distance between Registered email address...,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0
1,How to fix sleepimpl warning when ECS credenti...,Prerequisites X Ive searched for previous sim...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Slider doesnt work on touch devices,DescriptionSlider should work dragging in tou...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,add new labels,DescriptionAdd ui and logic to permanently ad...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,No lib sub folder in Boost folder,Hi I am following thishttps://www.mlpack.org/d...,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0
5,Add license notice to CLI,The CLI is missing the license notice. Theres ...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
6,Should show Powershell or AzureCLI code necess...,There is example output from Powershell and Az...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
7,tidboperator could not work with kubernetes 1.23,Bug ReportWhat version of Kubernetes are you ...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
8,Match Live,x Implement game logic x Calculate results ba...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
9,AngularBug Make current location widget more g...,If youve never submitted an issue to the SORMA...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [51]:
# check missing values in the dataset
print('The dataset has', df.isna().sum().sum(), 'missing values in test data.')
# check any duplicate records in the dataset
print('The dataset has', df.duplicated().sum(), 'duplicates in train data.')

The dataset has 1308 missing values in test data.
The dataset has 46411 duplicates in train data.


In [52]:
# remove missing values in the dataset
df.dropna(inplace= True)
# remove all duplicate records in the dataset
df.drop_duplicates(inplace= True)

In [53]:
# check missing values in the dataset
print('The dataset has', df.isna().sum().sum(), 'missing values in test data.')
# check any duplicate records in the dataset
print('The dataset has', df.duplicated().sum(), 'duplicates in train data.')

The dataset has 0 missing values in test data.
The dataset has 0 duplicates in train data.


In [54]:
texts = df["body"].tolist()
labels = df[["task","bug","documentation","duplicate","enhancement","good_first_issue","help_wanted","invalid","question","wontfix","gitalk","priority_medium","priority_high","feature_request","feature"]].values


In [55]:
# spliting dataset to train and test
from sklearn.model_selection import train_test_split
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)


In [56]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, f1_score, classification_report

# Vectorize the texts using TF-IDF
vectorizer = TfidfVectorizer()
train_vectors = vectorizer.fit_transform(train_texts)
test_vectors = vectorizer.transform(test_texts)


In [57]:
from sklearn.ensemble import RandomForestClassifier

In [58]:
# Train the Random Forest Classifier
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(train_vectors, train_labels)

RandomForestClassifier(random_state=42)

In [59]:
# Predict on test set
predictions = rf.predict(test_vectors)

In [87]:
from sklearn.metrics import accuracy_score, hamming_loss, f1_score, classification_report
rpt = classification_report(test_labels, predictions)

print(rpt)

              precision    recall  f1-score   support

           0       0.91      0.26      0.40       596
           1       0.85      0.80      0.82     17769
           2       0.93      0.14      0.25      1883
           3       0.40      0.04      0.08       461
           4       0.71      0.38      0.49     10110
           5       0.90      0.13      0.23      2919
           6       0.91      0.12      0.21      2422
           7       0.59      0.07      0.12       333
           8       0.76      0.05      0.09      1877
           9       0.59      0.04      0.07       739
          10       1.00      0.88      0.94        82
          11       0.00      0.00      0.00         3
          12       1.00      0.19      0.32        16
          13       0.88      0.07      0.14       592
          14       0.93      0.14      0.25      3434

   micro avg       0.82      0.46      0.59     43236
   macro avg       0.76      0.22      0.29     43236
weighted avg       0.82   

/Users/macbookpro/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/macbookpro/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [66]:
from sklearn.metrics import hamming_loss
print("Hamming score: {}".format(hamming_loss(y_test, predictions)))

Hamming score: 0.08833934110151731


In [86]:
from sklearn.metrics import accuracy_score, hamming_loss, f1_score, precision_score, recall_score

accuracy = accuracy_score(test_labels, predictions)
hl = hamming_loss(y_test, predictions)
f1 = f1_score(test_labels, predictions, average='micro')
p = precision_score(test_labels, predictions, average='micro')
r = recall_score(test_labels, predictions, average='micro')



print(f"RF - Accuracy: {accuracy}")
print(f"RF - Hamming Loss: {hl}")
print(f"RF - Precision: {p}")
print(f"RF - Recall: {r}")
print(f"RF - F1 score: {f1}")

RF - Accuracy: 0.46617486796881547
RF - Hamming Loss: 0.08833934110151731
RF - Precision: 0.820890608280517
RF - Recall: 0.4613285225275234
RF - F1 score: 0.5906950573044688
